# Lisp Interpreter

## Helper functions

In [1]:
(defn third
  [lst]
  (nth lst 2))

#'user/third

In [2]:
(defn fourth
  [lst]
  (nth lst 3))

#'user/fourth

## Main interpreter function

In [3]:
(defn $eval
  [expr env]
  
  (cond
    
    ; Check for variable reference
    (symbol? expr)
    (if (contains? env expr)
      (env expr)
      (throw (RuntimeException. (str "Unbound variable: " expr))))
    
    ; Check for special forms
    (list? expr)
    (case (first expr)
      
      nil
      ()
      
      QUOTE
      (second expr)
      
      IF
      (let [condition (second expr)
            then (third expr)
            else (fourth expr)]
        (if ($eval condition env)
          ($eval then env)
          ($eval else env)))
      
      LAMBDA
      nil
      
      LABEL
      nil
      
      ; Ordinary function application
      (let [eval-all-expr (map #($eval % env) expr)
            fun (first eval-all-expr)
            args (rest eval-all-expr)]
        (apply fun args)))
    
    ; Anything that is not symbol or a list evals to itself
    :else
    expr))

#'user/$eval

## Unit Tests

In [4]:
(require '[clojure.test :refer [is]])

nil

In [5]:
(is (= 42 ($eval 42 {})))
(is (= true ($eval true {})))
(is (= "hello" ($eval "hello" {})))

true

In [6]:
(is (= 5 ($eval 'X {'X 5})))
(is (= 7 ($eval 'C {'A 5, 'B 6, 'C 7})))

true

In [7]:
(is (= 'A ($eval '(QUOTE A) {})))
(is (= '(1 2 3) ($eval '(QUOTE (1 2 3)) {})))
(is (= 42 ($eval '(QUOTE 42) {})))

true

In [8]:
(is (= 2 ($eval '(IF 1 2 3) {})))
(is (= 3 ($eval '(IF nil 2 3) {})))

true

In [9]:
(is (= 2 ($eval '(ADD 1 1) {'ADD +})))
(is (= '(1 2 3 4)  ($eval '(CONS 1 (QUOTE (2 3 4))) {'CONS cons})))
(is (= 1 ($eval '(CAR X) {'CAR first, 'X '(1 2 3 4)})))

true